## 1. Final Project: AI Safety Officer  ##

### 1.1 The Business Problem ###
**Problem statement:** According to NIOSH, about 10% of hit by an object accidents will end up with fatality.


**Proposed solution:** 
If deplyed and pre trained properly, our model could be deployed alongside a camera inspecting people aproaching to a hazardous area/facility.

**Method:** 
Keras model will be optimized to recognizing the face bounds and detecting whether there is a safety helmet on the persons head.If no hardhat detected, the algorithm will sound a voice alarm.


### 1.2 The Data: VOC28 Worksite safety helmets dataset  ###
**5000 images availiable within the following link:** https://data.mendeley.com/datasets/9rcv8mm682/3




In [ ]:
!pip install tensorflow
!pip install keras
!pip install imutils
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install scipy

!pip install playsound
!pip install pygame

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=56bc503637048f3887f1c5b2e757f38959a4d370990dfbf760d747336624f461
  Stored in directory: /root/.cache/pip/wheels/ba/f8/bb/ea57c0146b664dca3a0ada4199b0ecb5f9dfcb7b7e22b65ba2
Successfully built playsound


### 1.2 Importing the necessary packages ###



In [ ]:

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

from playsound import playsound
from pygame import mixer
import time
from datetime import datetime
from timeit import default_timer as timer



### 1.3 OpenCV window  ###



In [ ]:
mixer.init()
sound = mixer.Sound('put_hat.wav')

def detect_and_predict_helmet(frame, faceNet, helmetNet):

	# Extract the dimensions of the camera frame and construct a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# Pass the blob through the network and detect the face(if availiable)
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# Initialize our list of faces, their locations,
	# and the list of predictions from our helmet network
	faces = []
	locs = []
	preds = []

	# Loop over the detected faces
	for i in range(0, detections.shape[2]):

		# Extract the confidence(proba) associated with each face detected
		confidence = detections[0, 0, i, 2]

		# Filter out weak detections by ensuring the confidence is greater than minth
		if confidence > 0.5:

			# Compute the (x, y)-coordinates of the bounding box for each face detected
			box = detections[0, 0, i, 3:7] * np.array([w-30, h-100, w+25, h])
			(startX, startY, endX, endY) = box.astype("int")

			# Ensure the bounding boxes fall within the dimensions of the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# Extract the face ROI, convert it from BGR to RGB channel
			# Resize it to 224x224, and preprocess it:
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# Add the face and bounding boxes to their respective lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# Only deliever a predictions if at least one face was detected:

	if len(faces) > 0:

		# For faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop:
		faces = np.array(faces, dtype="float32")
		preds = helmetNet.predict(faces, batch_size=32)

	# Return a 2-tuple of the face locations and their locations
	return (locs, preds)

# Load our serialized face detector model from disk:
prototxtPath = r"deploy.prototxt"
weightsPath = r"res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# Load our pretrained safety helmet classifier model from disk
helmetNet = load_model("hat_detector.model")

# Initialize the video stream
print( "starting video stream...")
vs = VideoStream(src=0).start()

# Loop over the frames from the video stream
while True:

	# Grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# Detect faces in the frame and determine if they are wearing a
	# safety helmet or not
	(locs, preds) = detect_and_predict_helmet(frame, faceNet, helmetNet)

	# Loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):

		# Unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(hat, withouthat) = pred

	# Sound a appropriate alarm in case the helmet is not found
		# start = 0        
		# if withouthat>hat:
		# 	  start = timer()
        
		label = "Safety Helmet" if hat > withouthat else "Put Your Safety Helmet" and sound.play() # and playsound('put_hat.wav')
                          
		# if start > 0.5: 
		# 	playsound('put_hat.wav')
		# 	time.sleep(3)
		# 	playsound('put_hat.wav')
		
		# Determine the class label and color of the bounding box and text
		color = (0, 150, 0) if label == "Safety Helmet" else (0, 116, 250)

		# Include the probability in the label
		label = "{}: {:.2f}%".format(label, max(hat, withouthat) * 100)

		# Display the label and bounding box rectangle on the output frame 
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# Show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# If the `q` key was pressed, break from the loopq
	if key == ord("q"):
		break

# Bit of cleanup
cv2.destroyAllWindows()
vs.stop()

NameError: ignored